In [48]:
pip install pdf2image pytesseract

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pdfplumber
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from pdf2image import convert_from_path
import pandas as pd
import re

In [2]:
def extract_text_from_image(page):
    """Manejo de PDFs escaneados usando OCR"""
    img = page.to_image(resolution=300)
    return pytesseract.image_to_string(img.original)

In [16]:
# Función para extraer preguntas del archivo PDF de preguntas

def extract_questions(pdf_path):
    
    questions = []
    community_id = os.path.basename(pdf_path).split("-")[0]
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            full_text = ""
            
            for page in pdf.pages:
                # Intenta extracción de texto normal
                text = page.extract_text()
                
                # Si falla la extracción normal, usa OCR
                if not text or len(text.strip()) < 50:
                    text = extract_text_from_image(page)
                
                full_text += text + "\n"

            # Buscar el inicio de las preguntas
            start_match = re.search(r"\n1[\.\-\)\s]", full_text)
            if start_match:
                full_text = full_text[start_match.start():]

            # Expresión regular para las preguntas
            question_pattern = r'''
            (\d+)[\.\-\)\s]*\s*       # Número de pregunta
            (.*?)\n                # Texto de la pregunta
            [aA]\)\s*(.*?)\n          # Opción A
            [bB]\)\s*(.*?)\n           # Opción B
            [cC]\)\s*(.*?)\n           # Opción C
            [dD]\)\s*(.*?)          # Opción D
            (?=\n\d+[\.\-\)\s]|$)        # Lookahead para siguiente pregunta o fin
            '''
            
            matches = re.findall(question_pattern, full_text, re.DOTALL | re.VERBOSE | re.IGNORECASE)

            for match in matches:
                question_id = match[0]
                question_text = match[1].strip()
                options = {
                    'A': match[2].strip(),
                    'B': match[3].strip(),
                    'C': match[4].strip(),
                    'D': match[5].strip()
                }
            
                    # Validación básica
                if int(question_id) <= 200:
                    questions.append({
                        'ComunidadID': community_id,
                        'PreguntaID': int(question_id),
                        'Pregunta': question_text,
                        **options
                    })
    
    except Exception as e:
        print(f"Error procesando {pdf_path}: {str(e)}")
    
    return questions

In [17]:
# Función para procesar todos los PDFs de la carpeta
def process_all_pdfs(pdf_folder):
    all_questions = []
    
    # Listar todos los archivos en la carpeta
    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith(".pdf") and "Preguntas" in pdf_file:
            pdf_path = os.path.join(pdf_folder, pdf_file)
            print(f"Procesando archivo: {pdf_file}")
            
            # Extraer solo las preguntas
            questions_data = extract_questions(pdf_path)

            # Agregar preguntas de este archivo a la lista global
            all_questions.extend(questions_data)
    
    # Guardar todos los datos extraídos en un CSV
    df = pd.DataFrame(all_questions)
    df.to_csv("preguntas.csv", index=False, encoding="utf-8")
    print("Archivo CSV con preguntas generado con éxito!")

In [18]:
# Carpeta donde se encuentran los PDFs
pdf_folder = "..\Examenes"

# Procesar todos los PDFs en la carpeta
process_all_pdfs(pdf_folder)

Procesando archivo: AND2021-Preguntas.pdf
Procesando archivo: AND2023-Preguntas.pdf
Procesando archivo: ARA2022-Preguntas.pdf
Procesando archivo: ARA2024-Preguntas.pdf
Procesando archivo: AS2024-Preguntas.pdf
Procesando archivo: BA2023-Preguntas.pdf
Procesando archivo: CA2024-Preguntas.pdf
Procesando archivo: CAN2019-Preguntas.pdf
Procesando archivo: CL2023-Preguntas.pdf
Procesando archivo: CLM2022-Preguntas.pdf
Procesando archivo: CLM2024-Preguntas.pdf
Procesando archivo: CM2024-Preguntas.pdf
Procesando archivo: CN2019-Preguntas.pdf
Procesando archivo: CV2022-Preguntas.pdf
Procesando archivo: EX2022-Preguntas.pdf
Procesando archivo: GA202301-Preguntas01.pdf
Procesando archivo: GA202302-Preguntas02.pdf
Procesando archivo: LR2021-Preguntas.pdf
Procesando archivo: LR2024-Preguntas.pdf
Procesando archivo: MA2019-Preguntas.pdf
Procesando archivo: MU2023-Preguntas.pdf
Procesando archivo: NV2022-Preguntas.pdf
Procesando archivo: NV2024-Preguntas.pdf
Procesando archivo: PV2022-Preguntas.pdf
P

In [ ]:
preguntas_file = pd.read_csv("preguntas.csv")

preguntas_file.head(20)


In [12]:
# Compruebo la cantidad de cuestionarios y su cantidad de preguntas
comunidad_counts = preguntas_file['ComunidadID'].value_counts()

print("ComunidadID únicas encontradas y su conteo de preguntas:")
print(comunidad_counts)

ComunidadID únicas encontradas y su conteo de preguntas:
PV2022      330
PV2023      221
AND2021     152
AND2023     151
EX2022      142
CN2019      130
MU2023      122
CM2024      121
NV2022      110
GA202302    102
NV2024      100
MA2019      100
LR2021      100
LR2024       96
CA2024       95
CV2022       70
CLM2022      69
AS2024       60
ARA2022      57
CLM2024      55
CL2023       48
ARA2024      12
GA202301     10
BA2023        8
Name: ComunidadID, dtype: int64


In [111]:
# Carpeta donde se encuentran los PDFs
pdf_path = "NV2022-Preguntas.pdf"
pdf_ara= extract_questions(pdf_path)

print(pdf_ara)


[{'ComunidadID': 'NV2022', 'PreguntaID': 195, 'Pregunta': 'E/2021, de 10 de septiembre, de la Consejera\nde Salud del Servicio Navarro de Salud-Osasunbidea)\n6 DE FEBRERO DE 2022\nNO PASE A LA HOJA SIGUIENTE\nMIENTRAS NO SE LE INDIQUE QUE PUEDE COMENZAR\n1) ¿Cómo se denomina a la probabilidad de que un individuo enfermo tenga un\ntest positivo?', 'A': 'Sensibilidad', 'B': 'Especificidad', 'C': 'Valor Predictivo Positivo', 'D': 'Valor Predictivo Negativo\n2) Una quemadura de grado 2 sufrida por un paciente durante su episodio de\nhospitalización, es un tipo de indicador:\na) Indicador basado en una proporción\nb) Indicador basado en el índice\nc) Indicador de estructura\nd) Indicador basado en un suceso centinela\n3) En relación a los planes de cuidados estandarizados, señale la respuesta\nincorrecta:\na) Se basan en que ante problemas comunes existen respuestas similares\nb) No presuponen una limitación a la individualización de la atención\nc) Es un proceso dinámico durante el tiempo 